### Initialize

In [7]:
from datetime import datetime
import json
import os
import sys

from ipywidgets import Dropdown
import pandas as pd
import requests

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))
from demo.shared.tools import get_base_url, get_root_path, TENANT_SUFFIX
import src.esh_objects as esh

base_url = get_base_url()
root_path = get_root_path()
example_name = sys.path[0].split(os.sep)[-2]

def execute_search_query(query):
    r = requests.post(f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[['firstName', 'lastName']]
            df.rename(columns={'firstName': 'First Name', 'lastName': 'Last Name'}, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())

c:\development\github\kfuerst\hana-enterprise-search-engine\demo\examples\simple\notebooks\..\..\..


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\development\\github\\kfuerst\\hana-enterprise-search-engine\\demo\\examples\\simple\\notebooks\\..\\..\\..\\..\\shared\\.config.json'

### Search for first name "Jane" and last name "Doe"

In [2]:
so = esh.EshObject(
    count=True,
    top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items= [
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property= esh.Property(property='lastName'),
                operator= esh.ComparisonOperator.Search,
                value= esh.StringValue(value='Doe')),
            esh.Comparison(
                property= esh.Property(property='firstName'),
                operator= esh.ComparisonOperator.Search,
                value= esh.StringValue(value='Jane'))]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))

ESH query statement: /$all?$top=1&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND lastName:Doe AND firstName:Jane'))
Total count: 2


,First Name,Last Name
0,Jane,Doe


### Search for first name "Jane"

In [3]:
so = esh.EshObject(
    count=True,
    top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items= [
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property= esh.Property(property='firstName'),
                operator= esh.ComparisonOperator.Search,
                value= esh.StringValue(value='Jane'))]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))

ESH query statement: /$all?$top=1&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND firstName:Jane'))
Total count: 2


,First Name,Last Name
0,Jane,Doe


### Fuzzy-search for first name "June"

In [5]:
so = esh.EshObject(
    count=True,
    top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items= [
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property= esh.Property(property='firstName'),
                operator= esh.ComparisonOperator.Search,
                value= esh.StringValue(
                    value='June',
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            )
        ]
    )
)

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))

ESH query statement: /$all?$top=1&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND firstName:June~0.7'))
Total count: 2


,First Name,Last Name
0,Jane,Doe


### Nested query

In [9]:
so = esh.EshObject(
    count=True,
    top=10,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items= [
            esh.ScopeComparison(values='Person'),
            esh.Expression(
                operator=esh.LogicalOperator.OR,
                items=[
                    esh.Expression(
                        operator=esh.LogicalOperator.ROW,
                        items=[
                            esh.Expression(
                                operator=esh.LogicalOperator.AND,
                                items= [
                                    esh.Comparison(
                                        property= esh.Property(property='lastName'),
                                        operator= esh.ComparisonOperator.Search,
                                        value= esh.StringValue(value='Doe')),
                                    esh.Comparison(
                                        property= esh.Property(property='firstName'),
                                        operator= esh.ComparisonOperator.Search,
                                        value= esh.StringValue(value='John'))
                                        ]
                            )
                        ]
                    ),
                    esh.Expression(
                        operator=esh.LogicalOperator.ROW,
                        items=[
                            esh.Expression(
                                operator=esh.LogicalOperator.AND,
                                items= [
                                    esh.Comparison(
                                        property= esh.Property(property='lastName'),
                                        operator= esh.ComparisonOperator.Search,
                                        value= esh.StringValue(value='Doe')),
                                    esh.Comparison(
                                        property= esh.Property(property='firstName'),
                                        operator= esh.ComparisonOperator.Search,
                                        value= esh.StringValue(value='Jane'))
                                        ]
                            )
                        ]
                    )
                ]
            )
        ]
    )
)

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))

ESH query statement: /$all?$top=10&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND ((((lastName:Doe AND firstName:John))) OR ((lastName:Doe AND firstName:Jane)))'))
Total count: 4


,First Name,Last Name
0,John,Doe
1,Jane,Doe
2,John,Doe
3,Jane,Doe
